In [3]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
dataset = load_breast_cancer()

In [5]:
dataset['data'].shape

(569, 30)

In [6]:
dataset['target'].shape

(569,)

In [7]:
def transform(i):
    return 1 if i == 1 else -1
dataset['target'] = [transform(i) for i in dataset['target']]

In [8]:
dataset['data'] = StandardScaler().fit_transform(dataset['data'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataset['data'], dataset['target'], train_size=0.9)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()

In [12]:
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([512, 30]),
 torch.Size([512]),
 torch.Size([57, 30]),
 torch.Size([57]))

In [14]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(30, 5)
        self.fc2 = nn.Linear(5, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))  
        return x

In [15]:
def forward_hook(module, input, output):
    # This function will be called during the forward pass
    print(f"Inside forward hook for {module.__class__.__name__}")
    print(f"Input: {input}")
    print(f"Output: {output}")

In [17]:
EPOCHS = 5

# Create an instance of the model
model = SimpleModel()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.HingeEmbeddingLoss()

model.relu.register_forward_hook(forward_hook)

# Losses
losses = []

# training loop
for _ in range(EPOCHS):
    output = model(X_train)
    loss = criterion(output, y_train.unsqueeze(1))
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/Users/atifadib/opt/anaconda3/envs/torch_env/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Inside forward hook for ReLU
Input: (tensor([[-0.3347, -0.4867,  0.2971,  0.1297, -0.4327],
        [-0.0593, -0.2575, -0.0641, -0.4556, -0.5293],
        [-0.0271, -0.4708, -0.1255,  0.4200, -0.8247],
        ...,
        [-0.3502, -0.6276, -0.1889,  0.6976, -0.3345],
        [-0.1764,  2.1522, -0.3437, -1.2393, -0.6755],
        [-0.4706, -0.0051,  1.2064, -0.5826,  0.0422]],
       grad_fn=<AddmmBackward0>),)
Output: tensor([[0.0000, 0.0000, 0.2971, 0.1297, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4200, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000, 0.6976, 0.0000],
        [0.0000, 2.1522, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.2064, 0.0000, 0.0422]], grad_fn=<ReluBackward0>)
Inside forward hook for ReLU
Input: (tensor([[-0.1557],
        [-0.2296],
        [-0.3470],
        [-0.2390],
        [ 0.0962],
        [-0.3077],
        [-0.2563],
        [-0.4314],
        [-0.4545],
        [-0.2559],
        [-

In [ ]:
losses

In [ ]:
y_hat = model(X_test)

In [ ]:
y_hat.detach().numpy().tolist()

In [ ]:
y_test